In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df2_WS3T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/pythonProject/segment2_WS3_has_duration.csv")

In [3]:
df2_WS3T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,914197|528561|809459,Assessment Outcome - Successful,Assessment Outcome - Successful,25/11/2019 11:12:41,ETONGSMURRAY,Faculty,6,N,2. Perform Assessment,Part 3 - Assessment Outcome,...,MCW,HDCW,Full Degree,AS,AS,Y,$DEC,N,0 days 0 hours 0 minutes 14 seconds,14.0


In [4]:
numOfWorkflowProcesses = df2_WS3T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic


In [5]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [6]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [7]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df2_WS3T[df2_WS3T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [8]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df2_WS3T[df2_WS3T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [9]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df2_WS3T[df2_WS3T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED']=='Release Offer']
        count_df_id += df_id['CASE_ID'].count()
        count_df_end_step += df_end_step['CASE_ID'].count()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [10]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df2_WS3T[df2_WS3T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [11]:
s2_t3=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end,'step':list_step_per_key})
s2_t3.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,step
0,1,246,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,{Assessment Outcome - Not Successful}
1,2,55,12 days 23 hours 35 minutes 47 seconds,0.0 days 0.0 hours 1.0 minutes 50.0 seconds,0.0%,"{Send for Assessment, AR - Satisfied - previou..."
2,3,783,2 days 0 hours 10 minutes 45 seconds,0.0 days 0.0 hours 2.0 minutes 15.0 seconds,32.14%,"{Send for Assessment, AR - Satisfied - previou..."
3,4,336,2 days 10 hours 25 minutes 8 seconds,0.0 days 0.0 hours 2.0 minutes 23.0 seconds,23.96%,"{Send for Assessment, AR - Satisfied - previou..."
4,5,37,7 days 23 hours 12 minutes 59 seconds,0.0 days 0.0 hours 3.0 minutes 44.0 seconds,15.68%,"{Send for Assessment, AR - Created - previous ..."
5,6,18,18 days 21 hours 54 minutes 54 seconds,7.0 days 11.0 hours 24.0 minutes 33.0 seconds,11.11%,{Assessment Outcome - Higher Preference Succes...
6,7,5,75 days 15 hours 48 minutes 49 seconds,0.0 days 0.0 hours 8.0 minutes 22.0 seconds,8.57%,"{Send for Assessment, Assessment Outcome - Suc..."
7,8,1,7 days 1 hours 36 minutes 9 seconds,7.0 days 1.0 hours 36.0 minutes 9.0 seconds,12.5%,"{Assessment Outcome - Successful, Assessment D..."
8,9,1,5 days 5 hours 11 minutes 43 seconds,5.0 days 5.0 hours 11.0 minutes 43.0 seconds,11.11%,"{Assessment Outcome - Successful, Assessment D..."
9,10,3,65 days 15 hours 2 minutes 45 seconds,53.0 days 1.0 hours 58.0 minutes 58.0 seconds,6.67%,"{Send for Assessment, Assessment Outcome - Suc..."


In [12]:
s2_t3.to_csv('segment2_WS3_table.csv', index=False)






